<a href="https://colab.research.google.com/github/polrgn/biomedical_clustering_topic_modeling/blob/main/_appendix_DBLBM_M3_CORD19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application of the DBLBM $\mathcal{M}_4$ to a corpus of COVID-19 publications

###DBLBM $\mathcal{M}_4$ functions

In [ ]:
import numpy as np

Function for 'cold' random initialization

In [ ]:
def cold_initialize(x,n,d,g):
  while True:
    z = np.zeros(shape=(n,g))
    for i in range(0,n):
      z[i,np.random.randint(g)] = 1

    w = np.zeros(shape=(d,g))
    for j in range(0,d):
      w[j,np.random.randint(g)] = 1
    
    floor = np.empty(shape=(1,g))
    floor[:] = 0
    pi = np.maximum(np.sum(z,axis=0)/n,floor)[0]
    rho = np.maximum(np.sum(w,axis=0)/d,floor)[0]

    if (sum(pi==0)==0):
      break

  print('initial z:\n',z[0:min(10,n),],'...')
  print('initial w:\n',w[0:min(10,d),],'...')
  print('initial pi',pi)
  print('initial rho',rho)
  return z,w,pi,rho

Function for 'warm' initialization.

In [ ]:
def warm_initialize(x,n,d,g):
  while True:
    km = MiniBatchKMeans(n_clusters=g, init='k-means++', n_init=10, 
                         batch_size=int(np.round(np.shape(x)[0]/10)))
    km_cluster_labels_row = km.fit_predict(x)
    z = np.zeros(shape=(n,g))
    for i in range(0,n):
      z[i,km_cluster_labels_row[i]] = 1

    km = MiniBatchKMeans(n_clusters=g, init='k-means++', n_init=10,
                         batch_size=int(np.round(np.shape(x)[1]/10)))
    km_cluster_labels_col = km.fit_predict(np.transpose(x))
    w = np.zeros(shape=(d,g))
    for j in range(0,d):
      w[j,km_cluster_labels_col[j]] = 1
    
    floor = np.empty(shape=(1,g))
    floor[:] = 0
    pi = np.maximum(np.sum(z,axis=0)/n,floor)[0]
    rho = np.maximum(np.sum(w,axis=0)/d,floor)[0]

    if (sum(pi==0)==0):
      break

  print('initial z:\n',z[0:min(10,n),],'...')
  print('initial w:\n',w[0:min(10,d),],'...')
  print('initial pi',pi)
  print('initial rho',rho)
  return z,w,pi,rho,km_cluster_labels_row

In [ ]:
# Get epsilon hat
def eps_hat_funct (z,w,x,n,d,g):
  vect_x_kk_zw = np.empty(shape=g)
  vect_x_kk_zw[:]= np.NaN
  for k in range(0,g):
    x_kk_zw = 0
    for i in range(0,n):
      for j in range(0,d):
        x_kk_zw = x_kk_zw + z[i,k]*w[j,k]*x[i,j]
    vect_x_kk_zw[k] = x_kk_zw

  term_1 = np.sum(np.abs(vect_x_kk_zw - np.sum(z,axis=0)*np.sum(w,axis=0)))

  sum_x_kl_zw = 0
  #counter = 0
  for k in range(0,g):
    for l in range(0,g):
      if l == k : 
        continue
      x_kl_zw = 0
      for i in range(0,n):
        for j in range(0,d):
          x_kl_zw = x_kl_zw + z[i,k]*w[j,l]*x[i,j]
      sum_x_kl_zw = sum_x_kl_zw + x_kl_zw
      #counter +=1
      #print(counter)

  eps_hat = max((term_1 + sum_x_kl_zw)/(n*d),0.01)
  return(eps_hat)

In [ ]:
# Get Aik's
def get_Aiks(z,w,x,pi,n,d,g):
  A = np.empty(shape=(n,g))
  A[:]= np.NaN
  for i in range(0,n):
    for k in range(0,g):
      xik_w = 0
      w_j = 0
      for j in range(0,d):
        xik_w = xik_w + w[j,k]*x[i,j]
        w_j = w_j + w[j,k]
      term1 = np.abs(xik_w- w_j)

      sum_x_il_w = 0
      for l in range(0,g):
        if l == k : 
          continue
        x_il_w = 0
        for j in range(0,d):
          x_il_w = x_il_w + w[j,l]*x[i,j]
        sum_x_il_w = sum_x_il_w + x_il_w
      
      A[i,k] = -(term1 + sum_x_il_w)
  return A

def update_z(z,A,n):
  z_update = np.copy(z)
  for i in range(0,n):
    z_update[i,:] = 0
    k_star = np.where(A[i,:]==np.max(A[i,:]))[0][0]
    z_update[i,int(k_star)] = 1
  return z_update

In [ ]:
def maximize_classlikelihood_wrt_z(z,w,x,pi,n,d,g):
  current_w = np.copy(w)
  current_z = np.copy(z)
  current_pi = np.copy(pi)

  while True:
    A=get_Aiks(current_z,current_w,x,current_pi,n,d,g)
    updated_z = update_z(current_z,A,n)
    floor = np.empty(shape=(1,g))
    floor[:] = 0
    updated_pi = np.maximum(np.sum(updated_z,axis=0)/n,floor)[0]
    
    diff_z = np.sum(np.abs(updated_z-current_z))/np.sum(current_z)
    diff_pi = np.sum(np.abs(updated_pi-current_pi))/np.sum(current_pi)
 
    current_z = updated_z
    current_pi = updated_pi
    
    if((diff_z<=0.01) and (diff_pi<=0.01)):
      break
      
  return current_z, current_pi
  

In [ ]:
# Get Bjl's
def get_Bjls(z,w,x,rho,n,d,g):
  B = np.empty(shape=(d,g))
  B[:]= np.NaN
  for j in range(0,d):
    for l in range(0,g):
      xlj_z = 0
      z_l = 0
      for i in range(0,n):
        xlj_z = xlj_z + z[i,l]*x[i,j]
        z_l = z_l + z[i,l]
      term1 = np.abs(xlj_z- z_l)

      sum_x_kj_z = 0
      for k in range(0,g):
        if k == l : 
          continue
        x_kj_z = 0
        for i in range(0,n):
          x_kj_z = x_kj_z + z[i,k]*x[i,j]
        sum_x_kj_z = sum_x_kj_z + x_kj_z

      B[j,l] = -(term1 + sum_x_kj_z)
  return B

def update_w(w,B,d):
  w_update = np.copy(w)
  for j in range(0,d):
    w_update[j,:] = 0
    l_star = np.where(B[j,:]==np.max(B[j,:]))[0][0]
    w_update[j,int(l_star)] = 1
  return w_update

In [ ]:
def maximize_classlikelihood_wrt_w(z,w,x,rho,n,d,g):
  current_w = np.copy(w)
  current_z = np.copy(z)
  current_rho = np.copy(rho)

  while True:
    B=get_Bjls(current_z,current_w,x,current_rho,n,d,g)
    updated_w = update_w(current_w,B,d)
    floor = np.empty(shape=(1,g))
    floor[:] = 0
    updated_rho = np.maximum(np.sum(updated_w,axis=0)/d,floor)[0]

    diff_w = np.sum(np.abs(updated_w-current_w))/np.sum(current_w)
    diff_rho = np.sum(np.abs(updated_rho-current_rho))/np.sum(current_rho)
 
    current_w = updated_w
    current_rho = updated_rho
    
    if((diff_w<=0.01) and (diff_rho<=0.01)):
      break
      
  return current_w, current_rho

In [ ]:
def maximize_classlikelihood(z,w,x,pi,rho,n,d,g):
  current_z = np.copy(z)
  current_w = np.copy(w)
  current_pi = np.copy(pi)
  current_rho = np.copy(rho)

  while True:
    print('\nMaximizing with respect to z\n')
    updated_z, updated_pi = maximize_classlikelihood_wrt_z(current_z,current_w,x,current_pi,n,d,g)
    print('\nMaximizing with respect to w\n')
    updated_w, updated_rho = maximize_classlikelihood_wrt_w(updated_z,current_w,x,current_rho,n,d,g)
    
    diff_z = np.sum(np.abs(updated_z-current_z))/np.sum(current_z)
    diff_w = np.sum(np.abs(updated_w-current_w))/np.sum(current_w)
    diff_pi = np.sum(np.abs(updated_pi-current_pi))/np.sum(current_pi)
    diff_rho = np.sum(np.abs(updated_rho-current_rho))/np.sum(current_rho)

    current_z = updated_z
    current_w = updated_w
    current_pi = updated_pi
    current_rho = updated_rho
    
    if((diff_z<=0.01) and (diff_w<=0.01) and (diff_pi<=0.01) and (diff_rho<=0.01)):
      break
    
  return current_z, current_w, current_pi, current_rho

In [ ]:
def compute_loglikelihood(z,w,x,pi,rho,eps,n,d,g):
  L_C = np.log(1-eps)*n*d
  for i in range(0,n):
    for k in range(0,g):
      add = z[i,k]*np.log(pi[k])
      if ~np.isnan(add):
        L_C = L_C + add
      for j in range(0,d):
        add = (np.log(eps)-np.log(1-eps))*z[i,k]*w[j,k]*np.abs(x[i,j]-1)
        if ~np.isnan(add):
          L_C= L_C + add
  
  for j in range(0,d):
    for l in range(0,g):
      add = w[j,l]*np.log(rho[l])
      if ~np.isnan(add):
        L_C = L_C + add

  for i in range(0,n):
    for j in range(0,d):
      for k in range(0,g):
        for l in range(0,g):
          if l == k : 
            continue
          add = (np.log(eps)-np.log(1-eps))*z[i,k]*w[j,l]*x[i,j]
          if ~np.isnan(add):
            L_C = L_C + add
  return L_C

In [ ]:
def get_predicted_lab(z,n):
  labels_pred = np.empty(shape=(n,1))
  labels_pred[:]= np.NaN
  for i in range(0,n):
    labels_pred[i,0] = np.where(z[i,:]==1)[0]
  return labels_pred

### Application of a sample of CORD-19 dataset

We import the libraries.

In [ ]:
import pandas as pd
import scipy.sparse
from sklearn.utils import shuffle
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.preprocessing import scale

In [ ]:
# Mounting Google drive where we save our data and embeddings
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


We read the pre-processed abstracts and obtain the binary document-term matrix.

In [ ]:
N = 10000
abstract_df = pd.read_csv('drive/MyDrive/Project_Graph/abstract_df_clean_stopwords_lang_abstrlen.csv',index_col=0)
abstract_df = abstract_df[0:N]
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features = 5000, max_df = 0.3, min_df=0.05)
tf_idf_vectorizer = vectorizer.fit(abstract_df.abstract)
tf_idf_matrix = vectorizer.transform(abstract_df.abstract)
scipy.sparse.save_npz('drive/MyDrive/Project_Graph/tfidf_abstract', tf_idf_matrix)

In [ ]:
tfidf_view = scipy.sparse.load_npz('drive/MyDrive/Project_Graph/tfidf_abstract.npz')
tfidf_view_array = np.squeeze(np.asarray(tfidf_view.todense()))
print('Sparsity TF-IDF',sum(sum(tfidf_view_array==0))/(np.shape(tfidf_view_array)[0]*np.shape(tfidf_view_array)[1]))

Sparsity TF-IDF 0.923709


In [ ]:
x_tfidf = np.copy(tfidf_view_array)
x_bin = np.copy(tfidf_view_array)
x_bin[x_bin != 0] = 1
np.unique(tfidf_view_array)
print('Sparsity TF-IDF',sum(sum(tfidf_view_array==0))/(np.shape(tfidf_view_array)[0]*np.shape(tfidf_view_array)[1]))

Sparsity TF-IDF 0.923709


We run the DBLM $\mathcal{M}_3$ on the binary document-term matrix and K-means on the TF-IDF representation.

In [ ]:
n = np.shape(x_bin)[0]
d = np.shape(x_bin)[1]
range_g = [3,4,5,6]
labels_array_dblbm = np.empty(shape=(n,len(range_g)))
labels_array_km = np.empty(shape=(n,len(range_g)))
loglikelihood_dblbm = []
silhouette_dblbm = []
ch_dblbm = []
db_dblbm = []
silhouette_km = []
ch_km = []
db_km = []

i = 0
for g in range_g:
  # cold initialization
  #z0,w0,pi0,rho0,eps0= cold_initialize(x,n,d,g)
  # warm initialization
  z0,w0,pi0,rho0,labels_km = warm_initialize(x_tfidf,n,d,g)
  
  optim_z, optim_w, optim_pi, optim_rho = maximize_classlikelihood(z0,w0,x_bin,pi0,rho0,n,d,g)
  optim_eps = eps_hat_funct(optim_z, optim_w, x_bin, n, d, g)
  labels_array_dblbm[:,i] = get_predicted_lab(optim_z,n)[:,0]
  labels_array_km[:,i] = labels_km
  loglikelihood_dblbm.append(compute_loglikelihood(optim_z, optim_w, x_bin, optim_pi, optim_rho, optim_eps,n,d,g))
  silhouette_km.append(metrics.silhouette_score(x_tfidf, labels_km))
  ch_km.append(metrics.calinski_harabasz_score(x_tfidf, labels_km))
  db_km.append(metrics.davies_bouldin_score(x_tfidf, labels_km))
  silhouette_dblbm.append(metrics.silhouette_score(x_bin, labels_km))
  ch_dblbm.append(metrics.calinski_harabasz_score(x_bin, labels_km))
  db_dblbm.append(metrics.davies_bouldin_score(x_bin, labels_km))
  i +=1
  

initial z:
 [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]] ...
initial w:
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]] ...
initial pi [0.3665 0.2218 0.4117]
initial rho [0.005 0.02  0.975]

Maximizing with respect to z


Maximizing with respect to w


Maximizing with respect to z


Maximizing with respect to w


Maximizing with respect to z


Maximizing with respect to w



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


initial z:
 [[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]] ...
initial w:
 [[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]] ...
initial pi [0.2269 0.2907 0.1885 0.2939]
initial rho [0.005 0.005 0.015 0.975]

Maximizing with respect to z


Maximizing with respect to w


Maximizing with respect to z


Maximizing with respect to w


Maximizing with respect to z


Maximizing with respect to w



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


initial z:
 [[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]] ...
initial w:
 [[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]] ...
initial pi [0.0418 0.1929 0.2717 0.2806 0.213 ]
initial rho [0.295 0.005 0.69  0.005 0.005]

Maximizing with respect to z


Maximizing with respect to w


Maximizing with respect to z


Maximizing with respect to w


Maximizing with respect to z


Maximizing with respect to w



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


initial z:
 [[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]] ...
initial w:
 [[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]] ...
initial pi [0.2063 0.0468 0.2534 0.0458 0.2354 0.2123]
initial rho [0.005 0.005 0.005 0.96  0.02  0.005]

Maximizing with respect to z


Maximizing with respect to w


Maximizing with respect to z


Maximizing with respect to w


Maximizing with respect to z


Maximizing with respect to w



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


In [ ]:
for i in range(0,len(range_g)):
  unique, counts = np.unique(labels_array_dblbm[:,i], return_counts=True)
  print('DBLBM','g=',range_g[i],dict(zip(unique, counts)))
  unique, counts = np.unique(labels_array_km[:,i], return_counts=True)
  print('KM','g=',range_g[i],dict(zip(unique, counts)))

results_km = pd.DataFrame({'n_cluster':range_g,'silhouette':silhouette_km,
                          'CH_score':ch_km, 'DB_score':db_km})
results_dblbm = pd.DataFrame({'n_cluster':range_g,'silhouette':silhouette_dblbm,
                          'CH_score':ch_dblbm, 'DB_score':db_dblbm,
                          'loglikelihood':loglikelihood_dblbm})

DBLBM g= 3 {0.0: 9503, 1.0: 497}
KM g= 3 {0.0: 3665, 1.0: 2218, 2.0: 4117}
DBLBM g= 4 {0.0: 8858, 1.0: 878, 2.0: 264}
KM g= 4 {0.0: 2269, 1.0: 2907, 2.0: 1885, 3.0: 2939}
DBLBM g= 5 {1.0: 8417, 3.0: 1113, 4.0: 470}
KM g= 5 {0.0: 418, 1.0: 1929, 2.0: 2717, 3.0: 2806, 4.0: 2130}
DBLBM g= 6 {0.0: 7808, 1.0: 559, 2.0: 549, 4.0: 131, 5.0: 953}
KM g= 6 {0.0: 2063, 1.0: 468, 2.0: 2534, 3.0: 458, 4.0: 2354, 5.0: 2123}


In [ ]:
results_dblbm

,n_cluster,silhouette,CH_score,DB_score,loglikelihood
0,3,0.004696,108.280410,7.861541,-538736.919746
1,4,-0.008979,97.244395,7.806006,-540480.519302
2,5,-0.002098,81.072013,7.644597,-540618.928707
3,6,-0.003652,74.197820,7.249360,-541395.135187


In [ ]:
results_km

,n_cluster,silhouette,CH_score,DB_score
0,3,0.015620,126.910269,7.710810
1,4,0.015550,109.096849,7.668895
2,5,0.018126,108.760701,6.761509
3,6,0.019995,103.202757,6.369907


We compute the ICL for the DBLM $\mathcal{M}_3$ outputs.

In [ ]:
ICL =[]
for i in range(0,len(range_g)):
  icl_g = results_dblbm.loc[i,'loglikelihood'] - 2*np.log(n)*(range_g[i]-1)/2 - np.log(n*d)/2
  ICL.append(icl_g)
results_dblbm['ICL'] = ICL
results_dblbm

,n_cluster,silhouette,CH_score,DB_score,loglikelihood,ICL
0,3,0.004696,108.280410,7.861541,-538736.919746,-538762.594756
1,4,-0.008979,97.244395,7.806006,-540480.519302,-540515.404652
2,5,-0.002098,81.072013,7.644597,-540618.928707,-540663.024397
3,6,-0.003652,74.197820,7.249360,-541395.135187,-541448.441218


In [ ]:
results_km.to_csv('drive/MyDrive/Project_Graph/results_km_m4.csv')
results_dblbm.to_csv('drive/MyDrive/Project_Graph/results_dblbm_m4.csv')

We extract the top words by cluster.

In [ ]:
def get_topwords3(data,k):
    countvectorizer = CountVectorizer(ngram_range=(1,2), max_features = 5000, max_df = 0.4, stop_words=list_stop_words)
    data_vectorized = countvectorizer.fit_transform(data)
    word_df = pd.DataFrame({'word': countvectorizer.get_feature_names(), 'count': np.asarray(data_vectorized.sum(axis=0))[0]})
    tfidfvectorizer = TfidfVectorizer(ngram_range=(1,1),vocabulary= countvectorizer.vocabulary_)
    word_df['idf'] = list(tfidfvectorizer.fit(data).idf_.flatten())
    word_df['custom_index'] = scale(word_df['count']*word_df['idf'])
    word_df = word_df.sort_values(by='custom_index', ascending=False)
    return word_df.head(k)

def get_topwords_bycluster(df,cluster_var,n_clust,n_topword):
  topwords_glob = pd.DataFrame()
  for i in range(0,n_clust):
        data = df[df[cluster_var]==i]['abstract']
        print('cluster',i)
        #print(get_topwords(data,15),'\n')
        #print(get_topwords2(data,15),'\n')
        topword_clust = get_topwords3(data,n_topword)
        print(topword_clust[['word','custom_index']],'\n')
        topword_clust['cluster'] = str(i)
        topwords_glob = pd.concat([topwords_glob,topword_clust[['cluster','word','custom_index']]],axis=0)
  return topwords_glob

list_stop_words = np.load("drive/MyDrive/Project_Graph/list_stopwords.npz")
list_stop_words = list(list_stop_words['arr_0'])

dblbm_labels = labels_array_dblbm[:,1]

In [ ]:
abstract_df_clusterized = abstract_df
abstract_df_clusterized['cluster_km'] = labels_array_km[:,2]
abstract_df_clusterized['cluster_dblbm'] = dblbm_labels

get_topwords_bycluster(abstract_df_clusterized,'cluster_km',5,15).to_csv('drive/MyDrive/Project_Graph/topword_clust_km_m4.csv')
get_topwords_bycluster(abstract_df_clusterized,'cluster_dblbm',4-1,15).to_csv('drive/MyDrive/Project_Graph/topword_clust_dblbm_m4.csv')


cluster 0
                  word  custom_index
2767              mers     17.564375
2812       middle east     14.166778
1453     east syndrome     12.618761
4627     syndrome mers     11.999554
62                ace2      9.769098
417            binding      8.697259
551               cell      8.015864
3986               rna      8.001317
3745  receptor binding      7.974709
4291          specific      7.410478
252           antibody      7.333876
3705               rbd      7.224636
244         antibodies      7.037267
4880           vaccine      6.769362
2160               igg      6.731265 

cluster 1
                 word  custom_index
699              cell     14.976859
4914          viruses     12.383557
2230           immune     10.853341
2236  immune response      9.701527
4064              rna      8.872637
2134             host      8.863139
1698       expression      8.662374
109          activity      8.602126
4006         response      8.180400
4343         specific     